In [37]:
import pandas as pd
import numpy as np
import random
import altair as alt
import plotly.graph_objects as go
import requests
import inspect
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotify_tools
import networkx as nx
import networkx.algorithms.community as nx_comm
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import plotly.express as px
import pyvis
from pyvis import network as net
from itertools import combinations
# from community import community_louvain
from copy import deepcopy

import config

In [38]:
# storing the credentials:
#
CLIENT_ID = config.client_id
CLIENT_SECRET = config.client_secret
my_username = "owenyyyyy"

# instantiating the client.  This 'sp' version of the client is used repeatedly below
# source: Max Hilsdorf (https://towardsdatascience.com/how-to-create-large-music-datasets-using-spotipy-40e7242cc6a6)
client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [39]:
creator_id = "rich6833spot"
playlist_id = "75OAYmyh848DuB16eLqBtk"

In [40]:
# playlist_tracks(user_id: String, playlist_id: String): json_dict
playlist_tracks = pd.DataFrame(sp.user_playlist_tracks("rich6833spot", "75OAYmyh848DuB16eLqBtk"))
playlist_tracks

,href,items,limit,next,offset,previous,total
0,https://api.spotify.com/v1/playlists/75OAYmyh8...,"{'added_at': '2023-09-15T17:20:27Z', 'added_by...",100,None,0,None,79
1,https://api.spotify.com/v1/playlists/75OAYmyh8...,"{'added_at': '2023-09-15T17:21:50Z', 'added_by...",100,None,0,None,79
2,https://api.spotify.com/v1/playlists/75OAYmyh8...,"{'added_at': '2023-09-15T17:22:14Z', 'added_by...",100,None,0,None,79
3,https://api.spotify.com/v1/playlists/75OAYmyh8...,"{'added_at': '2023-09-15T17:39:01Z', 'added_by...",100,None,0,None,79
4,https://api.spotify.com/v1/playlists/75OAYmyh8...,"{'added_at': '2023-09-15T17:39:42Z', 'added_by...",100,None,0,None,79
...,...,...,...,...,...,...,...
74,https://api.spotify.com/v1/playlists/75OAYmyh8...,"{'added_at': '2023-09-15T20:18:28Z', 'added_by...",100,None,0,None,79
75,https://api.spotify.com/v1/playlists/75OAYmyh8...,"{'added_at': '2023-09-15T20:18:41Z', 'added_by...",100,None,0,None,79
76,https://api.spotify.com/v1/playlists/75OAYmyh8...,"{'added_at': '2023-09-15T20:24:26Z', 'added_by...",100,None,0,None,79
77,https://api.spotify.com/v1/playlists/75OAYmyh8...,"{'added_at': '2023-09-15T20:24:33Z', 'added_by...",100,None,0,None,79


In [41]:
sample_track = playlist_tracks.iloc[0]["items"]["track"]
sample_track

{'album': {'album_type': 'album',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4ogvuDRerGhZfSf7TtzHlr'},
    'href': 'https://api.spotify.com/v1/artists/4ogvuDRerGhZfSf7TtzHlr',
    'id': '4ogvuDRerGhZfSf7TtzHlr',
    'name': "Israel Kamakawiwo'ole",
    'type': 'artist',
    'uri': 'spotify:artist:4ogvuDRerGhZfSf7TtzHlr'}],
  'available_markets': ['AR',
   'AU',
   'AT',
   'BE',
   'BO',
   'BR',
   'BG',
   'CA',
   'CL',
   'CO',
   'CR',
   'CY',
   'CZ',
   'DK',
   'DO',
   'DE',
   'EC',
   'EE',
   'SV',
   'FI',
   'FR',
   'GR',
   'GT',
   'HN',
   'HK',
   'HU',
   'IS',
   'IE',
   'IT',
   'LV',
   'LT',
   'LU',
   'MY',
   'MT',
   'MX',
   'NL',
   'NZ',
   'NI',
   'NO',
   'PA',
   'PY',
   'PE',
   'PH',
   'PL',
   'PT',
   'SG',
   'SK',
   'ES',
   'SE',
   'CH',
   'TW',
   'TR',
   'UY',
   'US',
   'GB',
   'AD',
   'LI',
   'MC',
   'ID',
   'JP',
   'TH',
   'VN',
   'RO',
   'IL',
   'ZA',
   'SA',
   'AE',
   'BH',
   'QA',

In [42]:
playlist_audio_features = spotify_tools.analyze_playlist(creator_id, playlist_id, sp) #Stores the resulting data frame as the variable: playlist_data_frame
playlist_audio_features.head(5) #Displays the first n rows of the data frame 

/Users/owenyaggy/code/EncodingMusic/spotify_tools.py:65: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



,artist,album,track_name,track_id,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Israel Kamakawiwo'ole,Facing Future,Somewhere Over The Rainbow_What A Wonderful World,25U7raB3ZSszayTYClh4hF,0.6630,0.1680,0,-13.725,1,0.0448,0.000057,0.3610,0.6780,85.005,308027,4
1,Queen,A Night At The Opera (Deluxe Remastered Version),Bohemian Rhapsody - Remastered 2011,7tFiyTwD0nx5a1eklYtX2J,0.3920,0.4020,0,-9.961,0,0.0536,0.000000,0.2430,0.2280,143.883,354320,4
2,VOCES8,Enchanted Isle,The Sun never says,2aAzKiCSR8Ine2vYEbbVaB,0.0726,0.0946,8,-21.337,1,0.0412,0.956000,0.0981,0.0361,75.089,290693,5
3,Michael Jackson,Dangerous,Heal the World,7woW97CfcWaKtuC6W5BP2K,0.5450,0.4850,9,-8.045,1,0.0485,0.000002,0.1040,0.1010,80.948,384307,4
4,Coldplay,Ghost Stories,A Sky Full of Stars,0FDzzruyVECATHXKHFs9eJ,0.5450,0.6750,6,-6.474,1,0.0279,0.001970,0.2090,0.1620,124.970,267867,4


In [43]:
feature_columns = ["danceability", "energy", "speechiness", "liveness", "instrumentalness", "valence", "danceability"]
def createRadarElement(row, feature_cols):
    return go.Scatterpolar(
        r = row[feature_cols].values.tolist(), 
        theta = feature_cols, 
        mode = 'lines', 
        name = row['track_name'])

def get_radar_plot(playlist_id, features_list, spotipy_client):
    current_playlist_audio_df = spotify_tools.get_audio_features_df(pd.DataFrame(spotipy_client.playlist_items(playlist_id)), spotipy_client)
    current_data = list(current_playlist_audio_df.apply(createRadarElement, axis=1, args=(features_list, )))  
    fig = go.Figure(current_data, )
    fig.show(renderer='iframe')
    fig.write_image(playlist_id + '.png', width=1200, height=800)
    
def get_radar_plots(playlist_id_list, features_list):
    for item in playlist_id_list:
        get_radar_plot(item, features_list, sp)
        
playlist_id = "1NppEwvZhkjeG3ZTYoOwVM"
get_radar_plot(playlist_id, feature_columns, sp)

/Users/owenyaggy/code/EncodingMusic/spotify_tools.py:40: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



ValueError: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido


In [ ]:
playlist_audio_features[(playlist_tracks['danceability'] < 0.4) and (playlist_tracks['intstrumentalness'] > 0.6)]